In [47]:
import pandas as pd
import numpy as np

rand = np.random.RandomState(42)
num_negatives = 100

def write_ratings(file, df):
    with open(file, 'w') as fp:
        for index, row in df.iterrows():
            fp.write(f'{row.userId}\t{row.uri}\t{1.0}\n')

def write_negatives(file, test_df, main_df):
    with open(file, 'w') as fp:
        for index, row in test_df.iterrows():
            seen_uris = set(main_df[main_df.userId == row.userId].uri)
            seen_uris.add(row.uri)
            
            negatives = main_df[~main_df.uri.isin(seen_uris)].uri
    
            sampled = '\t'.join([str(sample) for sample in rand.choice(negatives, num_negatives)])
            
            fp.write(f'({row.userId},{row.uri})\t{sampled}\n')

mr = pd.read_csv('../data/mindreader/ratings.csv')

# Remove unknown ratings
mr = mr[(mr.sentiment != 0) & mr.isItem]
mr.drop('sentiment', axis=1, inplace=True)

for col in ('uri', 'userId'):
    mr[col] = mr[col].astype('category')

columns = mr.select_dtypes(['category']).columns
mr[columns] = mr[columns].apply(lambda x: x.cat.codes)

test = mr.groupby('userId').apply(lambda df: df.sample(1))
print(test)
train = mr.drop(test.index)
print(train)

write_ratings('ml-1m.train.rating', train)
write_ratings('ml-1m.test.rating', test)
write_negatives('ml-1m.test.negative', test, mr)

print('So slow')

              Unnamed: 0  Unnamed: 0.1  userId   uri  isItem
userId                                                      
0      19419       19419         19419       0  2339    True
1      19832       19832         19832       1  2272    True
2      73158       73158         73158       2   927    True
3      77023       77023         77023       3  1939    True
4      850           850           850       4  1533    True
...                  ...           ...     ...   ...     ...
845    46213       46213         46213     845   744    True
846    41469       41469         41469     846  1177    True
847    36764       36764         36764     847   857    True
848    26189       26189         26189     848  1786    True
849    35885       35885         35885     849   536    True

[850 rows x 5 columns]
